In [1]:
import json
import pprint
import pandas as pd
import numpy as np
import os
import subprocess
import pexpect
import sys
import time
import tldextract
import re
import pickle
import urllib.request 

In [2]:
def dataframe_summary(df):
    print(df.shape)
    print(df.describe())
    print("\n")
    print(df.info())
    return df.head(100)
    
def dataframe_sample(dfaa):
    return dfaa.sample(n=10).head(30)

pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [3]:
df_source_features = pd.read_pickle('dfSubdomainSourceFeaturesBaseSamuel_comRankingFeatures_v2.pkl')
newRows = []
for index, row in df_source_features.iterrows():
    if('www.' in row['subdomain']):
        rowCopy = row.copy()
        ext = tldextract.extract(row['subdomain'])
        rowCopy['subdomain'] = ext.domain +'.'+ ext.suffix
        if(rowCopy['subdomain'] not in list(df_source_features['subdomain'])):
            newRows.append(rowCopy.values)
df_source_features = df_source_features.append(pd.DataFrame(newRows, columns=df_source_features.columns)).reset_index(drop=True)
        
        
        

In [32]:
df_source_features

,desinformacao_label,subdomain,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count,subdomain_tenMill_rank,subdomain_tenMill_open_page_rank
0,0,g1.globo.com,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,11,13,2136.0,5.57
1,0,www.uol.com.br,54.239.152.111,US,False,True,38.91,-77.03,16509,US,True,12,6,11962.0,5.05
2,0,www.correio24horas.com.br,204.199.44.210,BR,True,False,-19.92,-43.95,3549,US,False,11,3,51590.0,4.51
3,0,tribunaonline.com.br,35.201.90.53,US,False,True,39.11,-94.54,15169,US,True,14,2,3138498.0,3.19
4,0,www.opovo.com.br,104.21.52.3,US,False,True,37.33,-121.84,13335,US,True,9,3,25745.0,4.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1,abim.inf.br,51.81.104.180,US,False,True,38.96,-77.34,16276,US,True,13,1,NaN,NaN
210,1,sbtnews.com.br,34.102.135.52,US,False,True,39.11,-94.54,15169,US,True,14,2,5275099.0,2.82
211,1,jornalbrasilonline.com.br,172.217.15.83,US,False,True,38.98,-77.38,15169,US,True,12,0,NaN,NaN
212,1,xn--falandodegraa-tgb.com.br,142.251.33.211,US,False,True,38.98,-77.38,15169,US,True,18,9,NaN,NaN


In [31]:
df_source_features[~df_source_features['subdomain'].str.contains('.br|.com')]

,desinformacao_label,subdomain,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count,subdomain_tenMill_rank,subdomain_tenMill_open_page_rank
13,0,www.jornalcidade.net,172.67.158.179,US,False,True,37.77,-122.39,13335,US,True,9,2,4342172.0,2.93
35,1,pleno.news,170.82.174.10,BR,True,False,-23.55,-46.63,266444,BR,True,16,2,157586.0,4.25
39,1,www.elmundo.es,151.101.249.50,US,False,True,37.77,-122.39,54113,US,True,25,8,1829.0,5.62
70,1,tierrapura.org,104.26.13.133,US,False,True,37.33,-121.84,13335,US,True,9,2,3136592.0,3.19
76,1,mpr21.info,63.250.43.5,US,False,True,34.04,-118.43,22612,US,True,14,1,2931373.0,3.25
77,1,www.20minutos.es,54.230.161.97,US,False,True,40.74,-74.19,16509,US,True,17,2,5489.0,5.25
84,1,livraria.seminariodefilosofia.org,170.82.173.10,BR,True,False,-23.55,-46.63,266444,BR,True,9,2,702562.0,4.10
87,1,es.sott.net,172.67.72.133,US,False,True,37.77,-122.39,13335,US,True,9,1,1525312.0,3.59
89,1,myright.news,154.188.125.224,EG,False,False,30.06,31.24,8452,EG,True,13,0,6488823.0,2.70
98,1,midiasemmascara.net,162.241.61.99,US,False,True,40.22,-111.61,46606,US,True,16,1,NaN,NaN


In [5]:
dataframe_summary(df_source_features)

(214, 15)
       subdomain_tenMill_rank  subdomain_tenMill_open_page_rank
count            1.470000e+02                        147.000000
mean             1.877128e+06                          3.964014
std              2.321055e+06                          0.938795
min              9.000000e+01                          2.490000
25%              4.180750e+04                          3.180000
50%              4.339200e+05                          4.120000
75%              3.159304e+06                          4.565000
max              9.179591e+06                          7.260000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   desinformacao_label               214 non-null    object 
 1   subdomain                         214 non-null    object 
 2   subdomain_ip                      214 non-null    

,desinformacao_label,subdomain,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count,subdomain_tenMill_rank,subdomain_tenMill_open_page_rank
0,0,g1.globo.com,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,11,13,2136.0,5.57
1,0,www.uol.com.br,54.239.152.111,US,False,True,38.91,-77.03,16509,US,True,12,6,11962.0,5.05
2,0,www.correio24horas.com.br,204.199.44.210,BR,True,False,-19.92,-43.95,3549,US,False,11,3,51590.0,4.51
3,0,tribunaonline.com.br,35.201.90.53,US,False,True,39.11,-94.54,15169,US,True,14,2,3138498.0,3.19
4,0,www.opovo.com.br,104.21.52.3,US,False,True,37.33,-121.84,13335,US,True,9,3,25745.0,4.71
5,0,www.diariodolitoral.com.br,172.67.150.33,US,False,True,37.77,-122.39,13335,US,True,9,2,238745.0,4.18
6,0,www.zmnoticias.com.br,143.208.10.6,BR,True,False,-23.55,-46.63,28209,BR,True,14,1,NaN,NaN
7,0,www.estadaomatogrosso.com.br,64.225.27.79,US,False,True,38.91,-77.03,14061,US,True,11,3,NaN,NaN
8,0,www.folhape.com.br,104.21.48.71,US,False,True,37.33,-121.84,13335,US,True,9,4,145293.0,4.27
9,0,www.gazetadopovo.com.br,13.224.210.67,US,False,True,39.99,-75.14,16509,US,True,12,5,18758.0,4.82


In [6]:
df_env_features = pd.read_pickle('features_env_v1.pkl')

In [7]:
dataframe_summary(df_env_features)

(7146, 12)
            retweets          likes  initial_tweets  tweet_count_max  \
count    7146.000000    7146.000000     7146.000000     7.146000e+03   
mean      372.032326    1398.328296       91.600756     2.651567e+05   
std      4531.019018   19762.137200      754.983432     3.385045e+05   
min         0.000000       0.000000        1.000000     0.000000e+00   
25%         0.000000       0.000000        1.000000     8.322800e+04   
50%         2.000000       2.000000        2.000000     1.634510e+05   
75%        15.000000      67.000000        9.000000     3.174220e+05   
max    232628.000000  776532.000000    20037.000000     3.077916e+06   

       tweet_count_avg  followers_count_max  followers_count_avg  \
count     7.146000e+03         7.146000e+03         7.146000e+03   
mean      1.100409e+05         8.993120e+05         3.233824e+05   
std       1.128256e+05         2.405971e+06         7.913217e+05   
min       0.000000e+00         0.000000e+00         0.000000e+00   


,retweets,likes,initial_tweets,tweet_count_max,tweet_count_avg,followers_count_max,followers_count_avg,following_count_max,following_count_avg,verified_proportion,url,desinformacao_label
0,4.0,17.0,6,718820,144588.500000,12971449,2.174644e+06,614,241.000000,0.333333,https://g1.globo.com/am/amazonas/noticia/a-gente-sai-de-casa-com-a-sensacao-de-que-nao-vai-voltar-diz-agente-sobre-rotina-em-cadeia-no-am.ghtml,0
1,0.0,0.0,1,317422,317422.000000,1333445,1.333445e+06,163,163.000000,1.000000,https://www.uol.com.br/esporte/olimpiadas/ultimas-noticias/2021/08/04/aulas-galvao-ensina-andreoli-a-narrar-nomes-dificeis-de-atletas-olimpicos.htm,0
2,2.0,3.0,2,132680,67299.000000,12481,6.255000e+03,128,107.500000,0.000000,http://agorarn.com.br/chamada/apos-ataque-onibus-transporte-publico-de-natal-volta-funcionar-neste-domingo/,0
3,0.0,0.0,2,132680,67736.000000,12481,6.315500e+03,315,221.500000,0.000000,http://agorarn.com.br/cidades/governo-cidadao-garante-continuidade-de-investimentos-no-distrito-irrigado-do-baixo-acu/,0
4,0.0,0.0,7,132680,61114.714286,12481,3.895286e+03,2110,866.000000,0.000000,http://agorarn.com.br/destaquefotos/juiz-quebra-sigilo-bancario-de-empresa-que-prestou-servicos-a-campanha-de-fatima/,0
5,0.0,0.0,2,132680,132205.500000,12481,1.098150e+04,468,298.000000,0.000000,http://agorarn.com.br/destaques/nordestao-inaugura-seu-segundo-atacarejo-no-rn-nesta-quinta-feira/,0
6,0.0,0.0,4,132680,39521.500000,12481,3.574000e+03,1703,773.750000,0.000000,http://agorarn.com.br/politica/dinheiro-sonegado-seria-suficiente-para-tampar-rombo-da-previdencia-diz-styvenson/,0
7,84.0,62.0,74,873842,119939.081081,3997654,1.140963e+05,17121,2045.729730,0.054054,http://josiasdesouza.blogosfera.uol.com.br/2014/10/31/aecio-chama-aliados-para-reaparicao-na-quarta/,0
8,106.0,119.0,40,980382,129596.125000,3997654,1.172631e+05,55035,4134.750000,0.100000,http://josiasdesouza.blogosfera.uol.com.br/2015/06/21/dilma-e-pesadelo-do-qual-lula-nao-pode-acordar/,0
9,358.0,450.0,59,505006,136371.474576,3997654,3.179787e+05,15397,1999.016949,0.118644,http://josiasdesouza.blogosfera.uol.com.br/2016/06/24/lava-jato-agora-prepara-denuncias-contra-lula/,0


In [8]:
with open("content_features/dados/NoticiasFiltradas/filtradas_fake_titles.txt") as f:
    news_dict = []
    for news in f:
        news_dict.append(json.loads(news))
    df_content_features_false = pd.DataFrame.from_dict(news_dict)
    
#jornal21brasil.com.br tava como fonte apenas .com por isso nao dava match
df_content_features_false.loc[df_content_features_false['url'] == 'https://www.jornal21brasil.com.br/2019/12/congresso-corta-saude-e-educacao-para.html', 'fonte'] = 'jornal21brasil.com.br'

In [9]:
with open("content_features/dados/NoticiasFiltradas/filtradas_true_titles.txt") as f:
    news_dict = []
    for news in f:
        news_dict.append(json.loads(news))
    df_content_features_true = pd.DataFrame.from_dict(news_dict)
    

In [10]:
dataframe_summary(df_content_features_true)

(1181, 4)
                                                                                                                            url  \
count                                                                                                                      1181   
unique                                                                                                                     1180   
top     https://www.diariodolitoral.com.br/mundo/japao-ve-disseminacao-inedita-da-covid-19-e-recorde-de-casos-em-toquio/147888/   
freq                                                                                                                          2   

               fonte  \
count           1181   
unique            33   
top     g1.globo.com   
freq             288   

                                                                     Titulo  \
count                                                                  1181   
unique                                                 

,url,fonte,Titulo,features
0,https://g1.globo.com/politica/noticia/2021/06/24/governo-diz-que-bolsonaro-repassou-denuncias-sobre-compra-da-covaxin-a-pazuello.ghtml,g1.globo.com,Governo diz que Bolsonaro repassou denúncias sobre contrato da Covaxin a Pazuello,"{'Toxicity': 0.24297374, 'Threat': 0.2184854, 'Insult': 0.3228127, 'sentiment_score': -0.10000000149011612, 'sentiment_magnitude': 0.10000000149011612, 'sentiment': 'Neutral', 'exclamation_number': 0, 'uppercase_words_number': 0, 'hashtags_number': 0, 'Kincaid': 8.5, 'ARI': 10.1, 'Coleman-Liau': 13.1, 'Flesch Index': 57.0, 'Fog Index': 11.4, 'Lix': 59.2, 'SMOG-Grading': 10.7, 'n_characters': 69, 'n_words': 13, 'word_avg_lenght': 5.31, 'n_sentences': 1, 'sentence_avg_length': 13.0, 'n_short_s..."
1,https://www.uol.com.br/vivabem/noticias/redacao/2020/05/22/da-para-receber-cabeleireiro-e-manicure-em-casa-durante-a-pandemia-entenda.htm,www.uol.com.br,Dá para receber cabeleireiro e manicure em casa durante a pandemia? Entenda,"{'Toxicity': 0.28396788, 'Threat': 0.52348995, 'Insult': 0.26752764, 'sentiment_score': 0.20000000298023224, 'sentiment_magnitude': 0.4000000059604645, 'sentiment': 'Neutral', 'exclamation_number': 0, 'uppercase_words_number': 0, 'hashtags_number': 0, 'Kincaid': 7.4, 'ARI': 5.9, 'Coleman-Liau': 9.6, 'Flesch Index': 52.7, 'Fog Index': 12.4, 'Lix': 56.0, 'SMOG-Grading': 9.7, 'n_characters': 62, 'n_words': 12, 'word_avg_lenght': 5.17, 'n_sentences': 2, 'sentence_avg_length': 6.0, 'n_short_sente..."
2,https://www.correio24horas.com.br/noticia/nid/a-poupanca-e-uma-alternativa-de-investimento-na-pandemia/,www.correio24horas.com.br,A poupança é uma alternativa de investimento na pandemia?,"{'Toxicity': 0.012811388, 'Threat': 0.07751938, 'Insult': 0.013248383, 'sentiment_score': 0.0, 'sentiment_magnitude': 0.0, 'sentiment': 'Neutral', 'exclamation_number': 0, 'uppercase_words_number': 1, 'hashtags_number': 0, 'Kincaid': 10.2, 'ARI': 7.1, 'Coleman-Liau': 11.0, 'Flesch Index': 37.9, 'Fog Index': 12.5, 'Lix': 42.3, 'SMOG-Grading': 10.7, 'n_characters': 46, 'n_words': 9, 'word_avg_lenght': 5.11, 'n_sentences': 1, 'sentence_avg_length': 9.0, 'n_short_sentence': 0, 'n_long_sentence':..."
3,https://tribunaonline.com.br/associacao-medica-entra-com-acao-para-receitar-cloroquina-no-espirito-santo,tribunaonline.com.br,Associação Médica entra com ação para receitar cloroquina no Espírito Santo,"{'Toxicity': 0.033501644, 'Threat': 0.07751938, 'Insult': 0.02169453, 'sentiment_score': 0.0, 'sentiment_magnitude': 0.0, 'sentiment': 'Neutral', 'exclamation_number': 0, 'uppercase_words_number': 0, 'hashtags_number': 0, 'Kincaid': 5.2, 'ARI': 4.6, 'Coleman-Liau': 5.4, 'Flesch Index': 84.5, 'Fog Index': 11.3, 'Lix': 35.0, 'SMOG-Grading': 10.7, 'n_characters': 59, 'n_words': 15, 'word_avg_lenght': 3.93, 'n_sentences': 1, 'sentence_avg_length': 15.0, 'n_short_sentence': 0, 'n_long_sentence': ..."
4,https://www.correio24horas.com.br/noticia/nid/conheca-vini-boca-de-09-o-diabo-ozado-em-forma-de-guri-que-ja-mandou-o-corona-partir/,www.correio24horas.com.br,"Conheça Vini Boca de 09, o ‘diabo ozado’ em forma de guri que já mandou o corona partir","{'Toxicity': 0.68951297, 'Threat': 0.23167875, 'Insult': 0.71376973, 'sentiment_score': 0.30000001192092896, 'sentiment_magnitude': 0.30000001192092896, 'sentiment': 'Positive', 'exclamation_number': 0, 'uppercase_words_number': 0, 'hashtags_number': 0, 'Kincaid': 5.5, 'ARI': 4.2, 'Coleman-Liau': 2.8, 'Flesch Index': 89.6, 'Fog Index': 7.6, 'Lix': 19.0, 'SMOG-Grading': 3.0, 'n_characters': 65, 'n_words': 19, 'word_avg_lenght': 3.42, 'n_sentences': 1, 'sentence_avg_length': 19.0, 'n_short_sen..."
5,https://www.uol.com.br/nossa/noticias/afp/2020/05/18/cornavirus-famoso-monte-fuji-no-japao-ficara-fechado-este-verao.htm,www.uol.com.br,"Coronavírus: famoso Monte Fuji, no Japão, ficará fechado este verão","{'Toxicity': 0.035832137, 'Threat': 0.22077091, 'Insult': 0.029333333, 'sentiment_score': 0.10000000149011612, 'sentiment_magnitude': 0.10000000149

In [11]:
print(len(list(df_content_features_false['url'])))
print(len(list(df_content_features_true['url'])))
a = pd.merge(df_content_features_true,df_source_features, left_on='fonte', right_on='subdomain')
b = pd.merge(df_content_features_false,df_source_features, left_on='fonte', right_on='subdomain')
dfFeatures = pd.concat([a,b],ignore_index=True)
print(len(list(dfFeatures[dfFeatures['desinformacao_label']==1]['url'])))
print(len(list(dfFeatures[dfFeatures['desinformacao_label']==0]['url'])))
dfFeatures = dfFeatures.drop('fonte', 1)
dfFeatures = pd.merge(dfFeatures,df_env_features , left_on='url', right_on='url')
print(len(list(dfFeatures[dfFeatures['desinformacao_label_x']==1]['url'])))
print(len(list(dfFeatures[dfFeatures['desinformacao_label_x']==0]['url'])))
dfFeatures = dfFeatures.drop('desinformacao_label_y', 1)

dfFeatures = dfFeatures.rename(columns={'desinformacao_label_x': 'desinformacao_label'})

content_cols = list(dict(dfFeatures['features'][0]).keys())
extracted_content_features = dict()
for col in content_cols:
    extracted_content_features[col] = []
    
for index, row in dfFeatures.iterrows():
    row_content_features = dict(row['features'])
    for col in content_cols:
        extracted_content_features[col].append(row_content_features[col])

for col in content_cols:
    dfFeatures[col] = extracted_content_features[col]
    
dfFeatures = dfFeatures.drop('features',1)

dfFeatures = dfFeatures.infer_objects()
    
dfFeatures['desinformacao_label']= dfFeatures['desinformacao_label'].astype('bool')
dfFeatures['subdomain_ip_cc']= dfFeatures['subdomain_ip_cc'].astype('category')
dfFeatures['subdomain_as_cc']= dfFeatures['subdomain_as_cc'].astype('category')
dfFeatures['sentiment']= dfFeatures['sentiment'].astype('category')

    

292
1181
283
1189
240
754


<ipython-input-11-1d6135e45a88>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfFeatures = dfFeatures.drop('fonte', 1)
<ipython-input-11-1d6135e45a88>:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfFeatures = dfFeatures.drop('desinformacao_label_y', 1)
<ipython-input-11-1d6135e45a88>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfFeatures[col] = extracted_content_features[col]
<ipython-input-11-1d6135e45a88>:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfFeatures = dfFeatures.drop('features',1)


In [12]:
dataframe_summary(dfFeatures)

(994, 130)
       subdomain_ip_latitude  subdomain_ip_longitude  domain_route_hops  \
count             994.000000              994.000000         994.000000   
mean                7.976942              -71.275141          11.726358   
std                30.372807               34.667188           2.876366   
min               -30.040000             -122.390000           0.000000   
25%               -22.870000             -116.652500          11.000000   
50%                -3.720000              -51.230000          11.000000   
75%                37.770000              -43.950000          12.000000   
max                52.310000               31.240000          25.000000   

       domain_dns_caa_txt_count  subdomain_tenMill_rank  \
count                994.000000            9.100000e+02   
mean                   5.467807            6.350626e+05   
std                    4.315301            1.425072e+06   
min                    0.000000            9.000000e+01   
25%               

,url,Titulo,desinformacao_label,subdomain,subdomain_ip,subdomain_ip_cc,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_as_n,subdomain_as_cc,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count,subdomain_tenMill_rank,subdomain_tenMill_open_page_rank,retweets,likes,initial_tweets,tweet_count_max,tweet_count_avg,followers_count_max,followers_count_avg,following_count_max,following_count_avg,verified_proportion,Toxicity,Threat,Insult,sentiment_score,sentiment_magnitude,sentiment,exclamation_number,uppercase_words_number,hashtags_number,Kincaid,ARI,Coleman-Liau,Flesch Index,Fog Index,Lix,SMOG-Grading,n_characters,n_words,word_avg_lenght,n_sentences,sentence_avg_length,n_short_sentence,n_long_sentence,n_paragraphs,paragraph_avg_length,n_questions,n_passive_sentence,longest_sentence,shortest_sentence,n_verbs_to_be,n_verbs_auxiliary,n_conjunctions,%_conjunctions,n_pronouns,%_pronouns,n_prepositions,%_prepositions,n_nomilizations,%_nomilizations,funct,pronoun,ppron,i,we,you,shehe,they,ipron,article,verb,auxverb,past,present,future,adverb,preps,conj,negate,quant,number,swear,social,family,friend,humans,affect,posemo,negemo,anx,anger,sad,cogmech,insight,cause,discrep,tentat,certain,inhib,incl,excl,percept,see,hear,feel,bio,body,health,sexual,ingest,relativ,motion,space,time,work,achieve,leisure,home,money,relig,death,assent,nonfl,filler
0,https://g1.globo.com/politica/noticia/2021/06/24/governo-diz-que-bolsonaro-repassou-denuncias-sobre-compra-da-covaxin-a-pazuello.ghtml,Governo diz que Bolsonaro repassou denúncias sobre contrato da Covaxin a Pazuello,False,g1.globo.com,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,11,13,2136.0,5.57,19.0,186.0,15,401981,5.404287e+04,108641,8.170800e+03,4987,1140.200000,0.066667,0.242974,0.218485,0.322813,-0.1,0.1,Neutral,0,0,0,8.5,10.1,13.1,57.0,11.4,59.2,10.7,69,13,5.31,1,13.0,0,0,1,1.0,0,0,13,13,0,0,0,0.00,0,0.00,2,0.15,0,0.00,33.333333,16.666667,8.333333,0.0,0.0,8.333333,8.333333,0.000000,16.666667,8.333333,8.333333,0.000000,0.000000,8.333333,0.000000,0.000000,25.000000,8.333333,0.000000,0.000000,0.000000,8.333333,25.000000,0.000000,0.000000,8.333333,16.666667,8.333333,8.333333,0.000000,0.000000,0.000000,16.666667,0.000000,0.000000,8.333333,8.333333,0.000000,0.000000,8.333333,8.333333,8.333333,0.000000,8.333333,0.000000,8.333333,0.000000,0.000000,0.000000,8.333333,8.333333,0.000000,16.666667,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.333333,0.000000
1,https://g1.globo.com/politica/cpi-da-covid/noticia/2021/07/20/secretaria-do-ministerio-da-saude-sugeriu-a-portugal-atendimento-precoce-contra-a-covid.ghtml,Secretária do Ministério da Saúde sugeriu a Portugal 'atendimento precoce' contra a Covid,False,g1.globo.com,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,11,13,2136.0,5.57,41.0,239.0,8,26504,1.282625e+04,136618,1.744388e+04,1556,605.375000,0.125000,0.063448,0.223958,0.070534,0.3,0.3,Positive,0,0,0,9.8,7.8,8.8,53.1,11.4,41.0,10.7,72,16,4.50,1,16.0,0,0,1,1.0,0,0,16,16,0,0,0,0.00,0,0.00,4,0.25,0,0.00,38.461538,15.384615,15.384615,0.0,0.0,15.384615,15.384615,0.000000,15.384615,15.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.461538,0.000000,0.000000,0.000000,0.000000,0.000000,23.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.384615,7.692308,0.000000,0.000000,0.000000,0.000000,7.692308,0.000000,7.692308,0.000000,7.692308,7.692308,0.000000,15.384615,0.000000,15.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.384615,0.000000
2,https://g1.globo.com/df/distrito-federal/noticia/2021/08/02/vacinacao-contra-covid-19-2a-dose-da-pfizer-sera-aplicada-nesta-segunda-veja-locais.ghtml,Vacinação contra Covid-19: 2ª dose da Pfizer é aplicada nesta segunda; veja locais no DF,False,g1.globo.com,186.192.81.31,BR,True,False,-22.87,-42.35,28604,BR,True,

In [13]:
dfFeatures.dtypes

url                                   object
Titulo                                object
desinformacao_label                     bool
subdomain                             object
subdomain_ip                          object
subdomain_ip_cc                     category
subdomain_ip_is_brazil                  bool
subdomain_ip_is_us                      bool
subdomain_ip_latitude                float64
subdomain_ip_longitude               float64
subdomain_as_n                        object
subdomain_as_cc                     category
subdomain_ipcc_equal_ascc               bool
domain_route_hops                      int64
domain_dns_caa_txt_count               int64
subdomain_tenMill_rank               float64
subdomain_tenMill_open_page_rank     float64
retweets                             float64
likes                                float64
initial_tweets                         int64
tweet_count_max                        int64
tweet_count_avg                      float64
followers_

In [14]:
label_column_name = 'desinformacao_label'
numericalFeatures = []
boolFeatures = []
categorialFeatures = []
nonFeatures = []
for column in list(dfFeatures.columns):
    if(column) == label_column_name:
        continue
    elif(dfFeatures.dtypes[column].name == 'object'):
        nonFeatures.append(column)
    elif(dfFeatures.dtypes[column].name == 'category'):
        categorialFeatures.append(column)
    elif(dfFeatures.dtypes[column].name == 'bool'):
        boolFeatures.append(column)
    elif(dfFeatures.dtypes[column].name == 'int64' or dfFeatures.dtypes[column].name == 'float64'):
        numericalFeatures.append(column)
    else:
        raise("Alguma feature ta com tipo indefinido")


In [15]:
print(nonFeatures) #Verificar no olho principalmente essa pq uma feature nova inserida como object vai cair aqui
print('\n')
print(boolFeatures)
print('\n')
print(categorialFeatures)
print('\n')
print(numericalFeatures)

['url', 'Titulo', 'subdomain', 'subdomain_ip', 'subdomain_as_n']


['subdomain_ip_is_brazil', 'subdomain_ip_is_us', 'subdomain_ipcc_equal_ascc']


['subdomain_ip_cc', 'subdomain_as_cc', 'sentiment']


['subdomain_ip_latitude', 'subdomain_ip_longitude', 'domain_route_hops', 'domain_dns_caa_txt_count', 'subdomain_tenMill_rank', 'subdomain_tenMill_open_page_rank', 'retweets', 'likes', 'initial_tweets', 'tweet_count_max', 'tweet_count_avg', 'followers_count_max', 'followers_count_avg', 'following_count_max', 'following_count_avg', 'verified_proportion', 'Toxicity', 'Threat', 'Insult', 'sentiment_score', 'sentiment_magnitude', 'exclamation_number', 'uppercase_words_number', 'hashtags_number', 'Kincaid', 'ARI', 'Coleman-Liau', 'Flesch Index', 'Fog Index', 'Lix', 'SMOG-Grading', 'n_characters', 'n_words', 'word_avg_lenght', 'n_sentences', 'sentence_avg_length', 'n_short_sentence', 'n_long_sentence', 'n_paragraphs', 'paragraph_avg_length', 'n_questions', 'n_passive_sentence', 'longest_sentence

In [16]:
print("Diferenças na porcentagem do total em valores de features categoricas entre instancias True e False")
for i, feature in enumerate(categorialFeatures+boolFeatures):
    dfTrue = dfFeatures[dfFeatures[label_column_name]==0]
    dfFalse = dfFeatures[dfFeatures[label_column_name]==1]
    display(
        (
            (dfTrue[feature].value_counts()/len(dfTrue)*100)
            -
            (dfFalse[feature].value_counts()/len(dfFalse)*100)
        ).round(2).to_frame(feature +" (%)").T
    )

Diferenças na porcentagem do total em valores de features categoricas entre instancias True e False


,BR,DE,EG,ES,FR,NL,US
subdomain_ip_cc (%),42.98,-0.42,-1.67,-2.08,-2.92,-2.08,-33.81


,BR,CA,DE,EG,ES,FR,US
subdomain_as_cc (%),22.74,-2.5,-3.75,-1.67,-2.08,-2.92,-9.82


,Neutral,Positive,Negative,Mixed
sentiment (%),2.78,3.7,-4.37,-2.1


,False,True
subdomain_ip_is_brazil (%),-42.98,42.98


,False,True
subdomain_ip_is_us (%),33.81,-33.81


,True,False
subdomain_ipcc_equal_ascc (%),-16.9,16.9


In [17]:
dfFeatures_encoded = pd.get_dummies(dfFeatures, columns = categorialFeatures)

dfFeatures_encoded['subdomain_tenMill_rank'] = dfFeatures_encoded['subdomain_tenMill_rank'].replace(np.nan, 10000000)
dfFeatures_encoded['subdomain_tenMill_open_page_rank'] = dfFeatures_encoded['subdomain_tenMill_open_page_rank'].replace(np.nan, dfFeatures_encoded.mean()['subdomain_tenMill_open_page_rank'])

encodedFeatures = [feature for feature in dfFeatures_encoded.columns if feature not in nonFeatures+[label_column_name]]
encodedFeatures

<ipython-input-17-003bf6e55a85>:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfFeatures_encoded['subdomain_tenMill_open_page_rank'] = dfFeatures_encoded['subdomain_tenMill_open_page_rank'].replace(np.nan, dfFeatures_encoded.mean()['subdomain_tenMill_open_page_rank'])


['subdomain_ip_is_brazil',
 'subdomain_ip_is_us',
 'subdomain_ip_latitude',
 'subdomain_ip_longitude',
 'subdomain_ipcc_equal_ascc',
 'domain_route_hops',
 'domain_dns_caa_txt_count',
 'subdomain_tenMill_rank',
 'subdomain_tenMill_open_page_rank',
 'retweets',
 'likes',
 'initial_tweets',
 'tweet_count_max',
 'tweet_count_avg',
 'followers_count_max',
 'followers_count_avg',
 'following_count_max',
 'following_count_avg',
 'verified_proportion',
 'Toxicity',
 'Threat',
 'Insult',
 'sentiment_score',
 'sentiment_magnitude',
 'exclamation_number',
 'uppercase_words_number',
 'hashtags_number',
 'Kincaid',
 'ARI',
 'Coleman-Liau',
 'Flesch Index',
 'Fog Index',
 'Lix',
 'SMOG-Grading',
 'n_characters',
 'n_words',
 'word_avg_lenght',
 'n_sentences',
 'sentence_avg_length',
 'n_short_sentence',
 'n_long_sentence',
 'n_paragraphs',
 'paragraph_avg_length',
 'n_questions',
 'n_passive_sentence',
 'longest_sentence',
 'shortest_sentence',
 'n_verbs_to_be',
 'n_verbs_auxiliary',
 'n_conjunctio

In [33]:
len(encodedFeatures)

139

In [18]:
dfFeatures_encoded[encodedFeatures].head(40)

,subdomain_ip_is_brazil,subdomain_ip_is_us,subdomain_ip_latitude,subdomain_ip_longitude,subdomain_ipcc_equal_ascc,domain_route_hops,domain_dns_caa_txt_count,subdomain_tenMill_rank,subdomain_tenMill_open_page_rank,retweets,likes,initial_tweets,tweet_count_max,tweet_count_avg,followers_count_max,followers_count_avg,following_count_max,following_count_avg,verified_proportion,Toxicity,Threat,Insult,sentiment_score,sentiment_magnitude,exclamation_number,uppercase_words_number,hashtags_number,Kincaid,ARI,Coleman-Liau,Flesch Index,Fog Index,Lix,SMOG-Grading,n_characters,n_words,word_avg_lenght,n_sentences,sentence_avg_length,n_short_sentence,n_long_sentence,n_paragraphs,paragraph_avg_length,n_questions,n_passive_sentence,longest_sentence,shortest_sentence,n_verbs_to_be,n_verbs_auxiliary,n_conjunctions,%_conjunctions,n_pronouns,%_pronouns,n_prepositions,%_prepositions,n_nomilizations,%_nomilizations,funct,pronoun,ppron,i,we,you,shehe,they,ipron,article,verb,auxverb,past,present,future,adverb,preps,conj,negate,quant,number,swear,social,family,friend,humans,affect,posemo,negemo,anx,anger,sad,cogmech,insight,cause,discrep,tentat,certain,inhib,incl,excl,percept,see,hear,feel,bio,body,health,sexual,ingest,relativ,motion,space,time,work,achieve,leisure,home,money,relig,death,assent,nonfl,filler,subdomain_ip_cc_BR,subdomain_ip_cc_DE,subdomain_ip_cc_EG,subdomain_ip_cc_ES,subdomain_ip_cc_FR,subdomain_ip_cc_NL,subdomain_ip_cc_US,subdomain_as_cc_BR,subdomain_as_cc_CA,subdomain_as_cc_DE,subdomain_as_cc_EG,subdomain_as_cc_ES,subdomain_as_cc_FR,subdomain_as_cc_US,sentiment_Mixed,sentiment_Negative,sentiment_Neutral,sentiment_Positive
0,True,False,-22.87,-42.35,True,11,13,2136.0,5.57,19.0,186.0,15,401981,54042.866667,108641,8.170800e+03,4987,1140.200000,0.066667,0.242974,0.218485,0.322813,-0.1,0.1,0,0,0,8.5,10.1,13.1,57.0,11.4,59.2,10.7,69,13,5.31,1,13.0,0,0,1,1.0,0,0,13,13,0,0,0,0.00,0,0.00,2,0.15,0,0.00,33.333333,16.666667,8.333333,0.0,0.0,8.333333,8.333333,0.000000,16.666667,8.333333,8.333333,0.000000,0.000000,8.333333,0.000000,0.000000,25.000000,8.333333,0.000000,0.000000,0.000000,8.333333,25.000000,0.0,0.000000,8.333333,16.666667,8.333333,8.333333,0.000000,0.000000,0.000000,16.666667,0.000000,0.000000,8.333333,8.333333,0.000000,0.000000,8.333333,8.333333,8.333333,0.000000,8.333333,0.000000,8.333333,0.000000,0.000000,0.000000,8.333333,8.333333,0.000000,16.666667,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.333333,0.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,True,False,-22.87,-42.35,True,11,13,2136.0,5.57,41.0,239.0,8,26504,12826.250000,136618,1.744388e+04,1556,605.375000,0.125000,0.063448,0.223958,0.070534,0.3,0.3,0,0,0,9.8,7.8,8.8,53.1,11.4,41.0,10.7,72,16,4.50,1,16.0,0,0,1,1.0,0,0,16,16,0,0,0,0.00,0,0.00,4,0.25,0,0.00,38.461538,15.384615,15.384615,0.0,0.0,15.384615,15.384615,0.000000,15.384615,15.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.461538,0.000000,0.000000,0.000000,0.000000,0.000000,23.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.384615,7.692308,0.000000,0.000000,0.000000,0.000000,7.692308,0.000000,7.692308,0.000000,7.692308,7.692308,0.000000,15.384615,0.000000,15.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.384615,0.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,True,False,-22.87,-42.35,True,11,13,2136.0,5.57,0.0,0.0,1,48982,48982.000000,50911,5.091100e+04,1,1.000000,1.000000,0.041336,0.151318,0.027591,0.0,0.0,0,1,0,5.1,3.6,6.8,70.1,5.7,21.9,6.9,60,13,4.62,2,6.5,0,0,1,2.0,0,0,8,5,0,0,0,0.00,0,0.00,1,0.08,0,0.00,26.666667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,13.333333,0.000000,13.333333,0.000000,0.000000,13.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.333333,6.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [19]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif

In [23]:
dfFeatures_encoded_noNull = dfFeatures_encoded.dropna(axis=0, how="any") #Dps do replace nao altera nada, vou deixar pro futuro
X = dfFeatures_encoded_noNull[encodedFeatures]
y = dfFeatures_encoded_noNull[label_column_name]
res = dict(zip(encodedFeatures,
               mutual_info_classif(X, y, discrete_features=True)
               ))

for key in list(res.keys()):
    if("_avg" in key):
        res.pop(key, None)


sortedRes = dict(sorted(res.items(), key=lambda item: -item[1]))

In [24]:
sortedRes 

{'subdomain_tenMill_rank': 0.5472864745280056,
 'tweet_count_max': 0.5360957651282643,
 'followers_count_max': 0.5334550868581363,
 'subdomain_tenMill_open_page_rank': 0.5285259945804988,
 'following_count_max': 0.5086573643209686,
 'Insult': 0.382230487035075,
 'subdomain_ip_latitude': 0.3694940051302173,
 'subdomain_ip_longitude': 0.36949400513021724,
 'Toxicity': 0.32965717402377287,
 'likes': 0.31667045434580965,
 'domain_dns_caa_txt_count': 0.29613187471091007,
 'retweets': 0.28303275351835905,
 'initial_tweets': 0.24974207727817774,
 'verified_proportion': 0.2257311430486171,
 'Threat': 0.2098917576785162,
 'domain_route_hops': 0.19486294269804189,
 'Flesch Index': 0.19127937524407118,
 'Lix': 0.17174403067293698,
 'ARI': 0.12127908643763037,
 'Coleman-Liau': 0.12085941580705839,
 'Fog Index': 0.1050150584484921,
 'n_characters': 0.1000637451581376,
 'funct': 0.08799100222783368,
 'relativ': 0.0850014347677299,
 'Kincaid': 0.08002038321237354,
 'cogmech': 0.07234182691575225,
 's

In [ ]:
selector = SelectKBest(mutual_info_classif, k=10)
selector.fit(dfFeatures_encoded[encodedFeatures], dfFeatures_encoded[label_column_name])
cols = selector.get_support(indices=True)
bestFeatures_mutualInfo = dfFeatures_encoded[encodedFeatures].iloc[:,cols].columns
bestFeatures_mutualInfo


In [ ]:
selector = SelectKBest(f_classif, k=10)
selector.fit_transform(dfFeatures_encoded[encodedFeatures], dfFeatures_encoded[label_column_name])
cols = selector.get_support(indices=True)
bestFeatures_fClassif = dfFeatures_encoded[encodedFeatures].iloc[:,cols].columns
bestFeatures_fClassif

In [ ]:
# Create and fit selector
selector = SelectKBest(mutual_info_classif, k=10)
selector.fit(features_df, target)
# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
features_df_new = features_df.iloc[:,cols]

In [ ]:
import sys
from numpy import percentile, min, array, asfortranarray
from operator import itemgetter

#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn import tree
from math import log, fabs

In [ ]:
def load_rank_data_sparsem(file):
    X = []
    y = []

    lineid = 0
    for line in file:
        line = line.split("#")[0]
        spl = line.strip().split()
        label = int(spl[0])
        y.append(label)
        n = len(spl)
        for s in spl[2:]:
            tok = s.split(":")
            featureid = int(tok[0])
            value = float(tok[1])
            X.append([lineid, featureid, value])
        lineid += 1
    return (X, y)


def load_rank_data(file):
    X = []
    y = []
    feats = set()
    for line in file:
        line = line.split("#")[0]
        spl = line.strip().split()
        label = int(spl[0])
        y.append(label)
        n = len(spl)
        sv = {}
        for s in spl[2:]:
            tok = s.split(":")
            featureid = int(tok[0])
            feats.add(featureid)
            value = float(tok[1])
            sv[featureid] = value
        X.append(sv)
    return (X, y, feats)


def to_densem(X, feats):
    feat2index = {}
    index2feat = []
    i = 0
    for f in feats:
        index2feat.append(f)
        feat2index[f] = i
        i += 1
    newX = []
    for row in X:
        new_row = [0.0 for i in range(len(feat2index))]
        for (key, value) in row.items():
            index = feat2index[key]
            new_row[index] = value
        newX.append(new_row)

    return (asfortranarray(array(newX)), index2feat)

In [ ]:
file = open(sys.argv[1])
(X, y, feats) = load_rank_data(file)
file.close()

In [ ]:
(X_, index2feat) = to_densem(X, feats)

In [ ]:
y_ = array(y)

In [ ]:
print (X_.shape)
print (y_.shape)
print (X_.flags)
print (y_.flags)

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy")

clf.fit(X_, y_)
importances = clf.feature_importances_

print (importances)
print (index2feat)

#std = np.std([tree.feature_importances_ for tree in forest.estimators_],
#             axis=0)
#indices = np.argsort(importances)[::-1]

# Print the feature ranking
#print("Feature ranking:")

#for f in range(len(feats)):
#    print("%d. feature %d (%f)" % (f + 1, index2feat(indices[f]), importances[indices[f]]))
